In [19]:
# Install dependencies
!pip install networkx matplotlib pygraphviz

In [20]:
import os

import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
from IPython.display import Image
from networkx.drawing.nx_agraph import to_agraph
from random import choice

# Ensure the assets directory exists
os.makedirs('assets', exist_ok=True)

prefix = "aufgabe_02"
asset_dir = os.path.join('assets', prefix)
os.makedirs(asset_dir, exist_ok=True)

# Aufgabe 02

In [21]:
transactions = [
    ('R', 1, 'A'),
    ('W', 1, 'A'),
    ('R', 7, 'B'),
    ('R', 2, 'A'),
    ('W', 2, 'A'),
    ('R', 1, 'B'),
    ('W', 1, 'B'),
    ('R', 2, 'B'),
    ('W', 2, 'B'),
    ('R', 3, 'A'),
    ('W', 4, 'A'),
    ('R', 5, 'B'),
    ('R', 1, 'C'),
    ('R', 6, 'C'),
    ('W', 8, 'A'),
    ('W', 8, 'B')
]

graph = nx.DiGraph()

# Process the transactions to identify dependencies
last_read = {}
last_write = {}

for operation, transaction, item in transactions:
    if operation == 'R':
        if item in last_write and last_write[item] != transaction:
            # Dependency: last write to item -> current read of item
            graph.add_edge(last_write[item], transaction, label=f'W->{item}->R')
        last_read[item] = transaction
    elif operation == 'W':
        if item in last_read and last_read[item] != transaction:
            # Dependency: last read of item -> current write to item
            graph.add_edge(last_read[item], transaction, label=f'R->{item}->W')
        if item in last_write and last_write[item] != transaction:
            # Dependency: last write to item -> current write to item
            graph.add_edge(last_write[item], transaction, label=f'W->{item}->W')
        last_write[item] = transaction

# Check for cycles in the graph
try:
    cycles = nx.find_cycle(graph, orientation='original')
    cycle_exists = True
except nx.exception.NetworkXNoCycle:
    cycle_exists = False

# Determine a serial schedule if no cycles exist
serial_schedule = []
if not cycle_exists:
    serial_schedule = list(nx.topological_sort(graph))

# Visualize the graph using PyGraphViz
pos = to_agraph(graph)
# labels = nx.get_edge_attributes(graph, 'label')
# plt.figure(figsize=(12, 8))
# nx.draw(graph, pos, with_labels=True, node_size=2000, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=20)
# nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
# plt.title("Transaction Dependency Graph")
# plt.show()



serial_schedule, cycle_exists